In [1]:
import pandas as pd
import numpy as np

## Load training dataset

In [2]:
dataset_path = './dataset_train.csv'
dataset_train = pd.read_csv(dataset_path)

In [3]:
dataset_train.describe()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
count,522609.000000,5.236360e+05,523799.000000,523799.000000,449977.000000,5.220660e+05,523789.000000,489027.000000
mean,51875.055757,4.353965e+05,3.232519,2.436813,11.890710,9.267241e+05,41595.346178,1956.916771
std,29929.240099,7.155421e+05,1.145956,1.134782,766.747614,5.314372e+05,24504.754257,1818.780363
min,0.000000,0.000000e+00,1.000000,1.000000,0.000000,9.700000e+01,602.000000,100.000000
25%,23592.000000,2.050000e+05,3.000000,2.000000,0.140000,4.645375e+05,22407.000000,1288.000000
50%,52743.000000,3.150000e+05,3.000000,2.000000,0.220000,9.338935e+05,33647.000000,1719.000000
75%,78482.000000,4.750000e+05,4.000000,3.000000,0.450000,1.385839e+06,61032.000000,2332.000000
max,110138.000000,1.150000e+08,99.000000,113.000000,100000.000000,2.001125e+06,99901.000000,800900.000000


In [4]:
dataset_path = './dataset_val.csv'
dataset_val = pd.read_csv(dataset_path)

In [5]:
dataset_train = dataset_train[dataset_train["status"] == "sold"]
dataset_val = dataset_val[dataset_val["status"] == "sold"]

In [6]:
input_columns = ['bed', 'bath', 'house_size']
output_columns = ['price']

all_columns = input_columns + output_columns

In [7]:
cleaned_train = dataset_train[all_columns].dropna()
training_data = np.array(cleaned_train, dtype=np.float32)
x = cleaned_train[input_columns]
x = np.array(x, dtype=np.float32)
y = cleaned_train[output_columns]
y = np.array(y, dtype=np.float32)

cleaned_val = dataset_train[all_columns].dropna()
x_val = cleaned_val[input_columns]
x_val = np.array(x_val, dtype=np.float32)
y_val = cleaned_val[output_columns]
y_val = np.array(y_val, dtype=np.float32)

## Creating model

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [9]:
model = keras.Sequential( 
  [
    layers.Input(3),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1),
  ] 
) 

In [10]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

In [11]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
  x, y,
  validation_data=(x_val, y_val),
  epochs=3,
  batch_size=128,
)

Epoch 1/3
2999/2999 [==============================] - 27s 9ms/step - loss: 311608803328.0000 - mae: 175496.1406 - val_loss: 291679764480.0000 - val_mae: 163220.9688
Epoch 2/3
2999/2999 [==============================] - 26s 9ms/step - loss: 304923475968.0000 - mae: 166358.4844 - val_loss: 289108393984.0000 - val_mae: 162990.5781
Epoch 3/3
2999/2999 [==============================] - 26s 9ms/step - loss: 288775667712.0000 - mae: 175373.9219 - val_loss: 281748799488.0000 - val_mae: 167999.4844


In [14]:
model.save('model.h5')

In [13]:
model2 = keras.models.load_model('model.h5')